In [59]:
!pip install -q lightning

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [60]:
!pip install gdown


In [61]:
import gdown

# URL của file trên Google Drive (sửa lại URL phù hợp)
url = "https://drive.google.com/uc?id=197JiJUIXbXpZylimFqtBz9DeA_dn2BLa"
output = "/kaggle/working/data_17K.zip"
gdown.download(url, output, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=197JiJUIXbXpZylimFqtBz9DeA_dn2BLa
From (redirected): https://drive.google.com/uc?id=197JiJUIXbXpZylimFqtBz9DeA_dn2BLa&confirm=t&uuid=db7d22f4-ae83-45de-b193-2d938ffc765f
To: /kaggle/working/data_17K.zip


  0%|          | 0.00/517M [00:00<?, ?B/s]

  2%|▏         | 8.91M/517M [00:00<00:27, 18.4MB/s]

  4%|▍         | 22.5M/517M [00:00<00:10, 45.1MB/s]

  7%|▋         | 36.2M/517M [00:00<00:07, 67.3MB/s]

  9%|▉         | 48.8M/517M [00:00<00:05, 81.9MB/s]

 13%|█▎        | 69.2M/517M [00:00<00:03, 114MB/s] 

 18%|█▊        | 91.2M/517M [00:00<00:02, 143MB/s]

 22%|██▏       | 111M/517M [00:01<00:02, 158MB/s] 

 25%|██▌       | 132M/517M [00:01<00:02, 171MB/s]

 29%|██▉       | 152M/517M [00:01<00:02, 180MB/s]

 33%|███▎      | 171M/517M [00:01<00:01, 182MB/s]

 37%|███▋      | 191M/517M [00:01<00:01, 169MB/s]

 40%|████      | 209M/517M [00:01<00:01, 171MB/s]

 44%|████▍     | 226M/517M [00:01<00:01, 173MB/s]

 48%|████▊   

'/kaggle/working/data_17K.zip'

In [62]:
#import thư viện
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
import torchmetrics
import torch
import torchvision
from sklearn.model_selection import train_test_split
import os
import glob
import zipfile

In [63]:
# Đường dẫn tới file zip đã tải về
zip_path = "/kaggle/working/data_17K.zip"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/kaggle/working/data_17K")

In [64]:
# Kiểm tra nội dung thư mục
data_dir = "/kaggle/working/data_17K"
print(os.listdir(data_dir))

['data']


In [65]:
# Path to the validation dataset
val_path = "/kaggle/working/data_17K/data/validation/"

# Path to the training dataset
train_path = "/kaggle/working/data_17K/data/train/"

# List of filenames in the training dataset directory
list_ = [name for name in os.listdir(train_path)]

In [66]:
# Mapping each label name to a unique integer index
label_to_index = {}
for index, label in enumerate(list_):
    label_to_index[label] = index

def get_path(path, labels_list, label_to_index):
    """
    Generates a list of dictionaries with image details, each containing:
    - 'name': the label name
    - 'path': the file path of an image
    - 'label': the integer label corresponding to the name

    Args:
        path (str): Base directory path for image files.
        labels_list (list): List of label names.
        label_to_index (dict): Dictionary mapping label names to unique integers.

    Returns:
        list: List of dictionaries with details for each image.
    """
    paths = []
    for name in labels_list:
        # Get all .png file paths in the directory for the current label
        file_paths = glob.glob(f"{path}{name}/*.png")

        # Append each image's details to paths list
        for file_path in file_paths:
            paths.append({
                "name": name,
                "path": file_path,
                "label": label_to_index[name]
            })
    return paths

# Retrieve paths and labels for training and validation datasets
train_paths = get_path(train_path, list_, label_to_index)
val_paths = get_path(val_path, list_, label_to_index)

In [67]:
class DataSet(torch.utils.data.Dataset):
    """
    Custom dataset class for loading and transforming images with labels.
    """

    def __init__(self, features, new_size=(224, 224)):
        """
        Initializes the dataset.

        Args:
            features (list): List of dictionaries containing image paths and labels.
            new_size (tuple): Desired size for resizing images. Default is (224, 224).
        """
        super(DataSet, self).__init__()
        self.features = features  # Store dataset features (paths and labels)
        self.resize_transform = torchvision.transforms.Resize(new_size)  # Resize transformation

    def __len__(self):
        """
        Returns:
            int: Number of samples in the dataset.
        """
        return len(self.features)

    def __getitem__(self, idx):
        """
        Retrieves and processes a sample from the dataset by index.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            dict: Dictionary with the transformed image tensor and label.
        """
        paths = self.features[idx]["path"]  # Image file path
        image = torchvision.io.read_image(paths) / 255.0  # Load and normalize image to [0, 1]
        image = self.resize_transform(image)  # Resize image to target dimensions
        label = torch.tensor(self.features[idx]["label"])  # Convert label to tensor
        return {"image": image, "label": label}

# Initialize the dataset with validation paths
Data = DataSet(val_paths)

# Example usage
next(iter(Data))['image'].shape

torch.Size([3, 224, 224])

In [68]:
class DataModule(L.LightningDataModule):
    """
    PyTorch Lightning data module for loading and managing datasets.
    Handles training, validation, and test data loaders.
    """
    def __init__(self, batch_size, num_workers, train_path, val_path):
        """
        Initializes the data module with batch size, number of workers,
        and paths to training and validation data.
        Args:
            batch_size (int): Number of samples per batch for data loaders.
            num_workers (int): Number of subprocesses for data loading.
            train_path (str): Directory path to training data.
            val_path (str): Directory path to validation data.
        """
        super(DataModule, self).__init__()
        self.train_path = train_path
        self.val_path = val_path
        self.batch_size = batch_size
        self.num_workers = num_workers

    def setup(self, stage=None):
        """
        Sets up the datasets for training, validation, and testing.
        Splits training data for validation and test sets.
        Args:
            stage (str, optional): The stage for setup (e.g., 'fit' or 'test').
        """
        train_paths = get_path(self.train_path, list_, label_to_index)  # Get training paths
        val_paths = get_path(self.val_path, list_, label_to_index)      # Get validation paths
        train_paths, test_paths = train_test_split(train_paths, test_size=0.2, random_state=0)  # Split for testing
        # Create datasets for train, validation, and test sets
        self.train_dataset = DataSet(train_paths)
        self.val_dataset = DataSet(val_paths)
        self.test_dataset = DataSet(test_paths)

    def train_dataloader(self):
        """
        Returns:
            DataLoader: DataLoader for training set with shuffling enabled.
        """
        return torch.utils.data.DataLoader(self.train_dataset, shuffle=True,
                                           batch_size=self.batch_size, num_workers=self.num_workers)

    def val_dataloader(self):
        """
        Returns:
            DataLoader: DataLoader for validation set with shuffling disabled.
        """
        return torch.utils.data.DataLoader(self.val_dataset, shuffle=False,
                                           batch_size=self.batch_size, num_workers=self.num_workers)

    def test_dataloader(self):
        """
        Returns:
            DataLoader: DataLoader for test set with shuffling disabled.
        """
        return torch.utils.data.DataLoader(self.test_dataset, shuffle=False,
                                           batch_size=self.batch_size, num_workers=self.num_workers)

# Instantiate and set up the data module
data_module = DataModule(32, 2, train_path, val_path)
data_module.setup()

In [69]:
class Model(L.LightningModule):
    """
    A PyTorch Lightning model that uses a ResNet152 backbone for feature extraction
    and a linear classifier for classification. Includes methods for training, validation,
    and testing steps.
    """
    def __init__(self, is_train=False, learning_rate=1e-3, num_target_classes=4):
        """
        Initializes the model with a feature extractor, classifier, and training settings.
        Args:
            is_train (bool): Whether to set the feature extractor to training mode.
            learning_rate (float): The learning rate for the optimizer.
            num_target_classes (int): Number of target classes for classification.
        """
        super(Model, self).__init__()
        # Load the pretrained ResNet152 model
        backbone = torchvision.models.resnet152(weights="DEFAULT")
        # Number of filters in the final fully connected layer of ResNet152
        num_filters = backbone.fc.in_features
        # Remove the last classification layer from ResNet152
        layers = list(backbone.children())[:-1]
        # Define the feature extractor and classifier
        self.feature_extractor = torch.nn.Sequential(*layers)
        self.classifier = torch.nn.Linear(num_filters, num_target_classes)
        # Model training parameters
        self.is_train = is_train
        self.learning_rate = learning_rate
        # Loss function and metric
        self.loss = torch.nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.classification.Accuracy(task="multiclass", num_classes=num_target_classes)

    def forward(self, x):
        """
        Defines the forward pass for the model.
        Args:
            x (Tensor): Input tensor (batch of images).
        Returns:
            Tensor: Output tensor (logits for each class).
        """
        # Set feature extractor to evaluation mode
        self.feature_extractor.eval()
        # Extract features
        if self.is_train:
            representations = self.feature_extractor(x).flatten(1)
        else:
            # Disable gradient calculations for inference
            with torch.no_grad():
                representations = self.feature_extractor(x).flatten(1)
        # Classify the representations
        x = self.classifier(representations)
        return x

    def training_step(self, batch, batch_idx):
        """
        Training step for a batch.
        Args:
            batch (dict): Batch of data, containing images and labels.
            batch_idx (int): Index of the batch.
        Returns:
            Tensor: Training loss for the batch.
        """
        x = batch["image"]
        y = batch["label"]
        # Forward pass
        y_pred = self(x)
        # Calculate loss and accuracy
        loss = self.loss(y_pred, y)
        acc = self.accuracy(y_pred, y)
        # Log metrics
        self.log_dict({'train_loss': loss, 'train_acc': acc}, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        """
        Validation step for a batch.
        Args:
            batch (dict): Batch of data, containing images and labels.
            batch_idx (int): Index of the batch.
        Returns:
            Tensor: Validation loss for the batch.
        """
        x = batch["image"]
        y = batch["label"]
        # Forward pass
        y_pred = self(x)
        # Calculate loss and accuracy
        val_loss = self.loss(y_pred, y)
        val_acc = self.accuracy(y_pred, y)
        # Log metrics
        self.log_dict({'val_loss': val_loss, 'val_acc': val_acc}, prog_bar=True)
        return val_loss

    def test_step(self, batch, batch_idx):
        """
        Test step for a batch.
        Args:
            batch (dict): Batch of data, containing images and labels.
            batch_idx (int): Index of the batch.
        Returns:
            Tensor: Test loss for the batch.
        """
        x = batch["image"]
        y = batch["label"]
        # Forward pass
        y_pred = self(x)
        # Calculate loss and accuracy
        test_loss = self.loss(y_pred, y)
        test_acc = self.accuracy(y_pred, y)
        # Log metrics
        self.log_dict({'test_loss': test_loss, 'test_acc': test_acc}, prog_bar=True)
        return test_loss

    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        """
        Predict step for a batch, used for generating predictions.
        Args:
            batch (dict): Batch of data, containing images.
            batch_idx (int): Index of the batch.
            dataloader_idx (int, optional): Index of the dataloader (for multi-dataloader scenarios).
        Returns:
            Tensor: Predicted labels for the batch.
        """
        x = batch["image"]
        # Forward pass to get predictions
        logits = self(x)
        # Get the predicted labels
        preds = torch.argmax(logits, dim=1)
        return preds

    def configure_optimizers(self):
        """
        Sets up the optimizer for the model.
        Returns:
            Optimizer: Adam optimizer with the specified learning rate.
        """
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [70]:
model_checkpoint = ModelCheckpoint( 
    dirpath='/kaggle/working/checkpoint/',
    monitor="val_acc",
    verbose=True,
    mode="max",
    save_top_k=1,
    filename="best_model" )


early_stopping = EarlyStopping(
    monitor="val_acc",
    mode="max",
    min_delta=1e-4,
    patience=5 )
callbacks = [model_checkpoint,early_stopping]

# Instantiate the model and DataModule
model = Model()
data_module = DataModule(128, 2, train_path, val_path)

# Create the PyTorch Lightning Trainer
trainer = L.Trainer(
    max_epochs=10,
    detect_anomaly=True,
    callbacks=callbacks
)

INFO: Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO: You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs


In [71]:
# Start training
trainer.fit(model, datamodule=data_module)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO: 
  | Name              | Type               | Params | Mode 
-----------------------------------------------------------------
0 | feature_extractor | Sequential         | 58.1 M | train
1 | classifier        | Linear             | 8.2 K  | train
2 | loss              | CrossEntropyLoss   | 0      | train
3 | accuracy          | MulticlassAccuracy | 0      | train
-----------------------------------------------------------------
58.2 M    Trainable params
0         Non-trainable params
58.2 M    Total params
232.608   Total estimated model params size (MB)
425       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 90: 'val_acc' reached 0.64415 (best 0.64415), saving model to '/kaggle/working/checkpoint/best_model-v1.ckpt' as top 1


...

INFO: Epoch 7, global step 720: 'val_acc' reached 0.70595 (best 0.70595), saving model to '/kaggle/working/checkpoint/best_model-v1.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 8, global step 810: 'val_acc' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 9, global step 900: 'val_acc' was not in top 1
INFO: `Trainer.fit` stopped: `max_epochs=10` reached.


In [72]:
import os

checkpoint_dir = '/kaggle/working/checkpoint/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)  # Tạo thư mục nếu nó không tồn tại


In [73]:
checkpoint_files = os.listdir(checkpoint_dir)
print("Checkpoint files:", checkpoint_files)

Checkpoint files: ['best_model.ckpt', 'epoch=9-step=900.ckpt', 'best_model-v1.ckpt']


In [74]:
best_model_path = os.path.join(checkpoint_dir, 'best_model.ckpt')
if os.path.isfile(best_model_path):
    best_model = Model.load_from_checkpoint(best_model_path)
    print("Model loaded successfully from", best_model_path)
else:
    print(f"Checkpoint file not found at {best_model_path}")


Model loaded successfully from /kaggle/working/checkpoint/best_model.ckpt


In [75]:
# Tải mô hình tốt nhất từ checkpoint
best_model = Model.load_from_checkpoint("checkpoint/best_model.ckpt")

In [76]:
from torch.utils.data import DataLoader

# Khởi tạo và thiết lập DataModule
data_module = DataModule(batch_size=128, num_workers=2, train_path=train_path, val_path=val_path)
data_module.setup('test')

# Tạo DataLoader cho tập dữ liệu kiểm tra
test_loader = data_module.test_dataloader()

# Kiểm tra mô hình trên tập dữ liệu kiểm tra
test_results = trainer.test(model, dataloaders=test_loader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.6991273760795593     │
│         test_loss         │    0.7362668514251709     │
└───────────────────────────┴───────────────────────────┘

In [77]:
# Dự đoán nhãn trên test_loader
test_predictions = trainer.predict(best_model, dataloaders=test_loader)


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [79]:

# Chuyển đổi các batch thành một danh sách nhãn dự đoán duy nhất
all_predictions = torch.cat(test_predictions).cpu().numpy()
print("Predictions:", all_predictions)


Predictions: [1 1 1 ... 1 1 2]
